In [1]:
# https://keras.io/examples/vision/mnist_convnet/
# https://www.kaggle.com/code/shivanisheth2911/lstm-keras-on-mnist

In [2]:
import keras
from keras import layers
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Dataset

In [3]:
input_shape = (28, 28, 1)
num_classes = 10

In [4]:
# Dataset Size
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Make Models

In [5]:
def make_cnn(input_shape, num_classes):
    model = keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dropout(0.5),
            layers.Dense(num_classes, activation="softmax"),
        ]
    )

    model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

    return model

## Experiment

In [6]:
def fit_and_validate(model, model_name, epochs, x_train, y_train, x_test, y_test, df):
    batch_size = 128

    start_time = time.time()
    model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
    time_to_train = time.time() - start_time

    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    start_time = time.time()
    model.predict(x_test)
    time_to_predict = time.time() - start_time

    df.loc[len(df)] = [model_name, batch_size, epochs, len(x_train), time_to_train, 0, time_to_predict, score[1]]
    return df

In [14]:
# Action Space
all_epochs = [2, 5]
all_models = {
    "CNN": make_cnn(input_shape, num_classes)
}
all_dataset_sizes = [i/100 for i in range(25, 105, 5)]

In [15]:
# Data Collection
df = pd.DataFrame(columns=['model', 'batch_size', 'epochs', 'dataset_size', 'time_to_train', 'time_to_retrain', 'time_to_predict', 'accuracy'])

In [ ]:
for epochs in all_epochs:
    for model in all_models:
        for dataset_size in all_dataset_sizes:
            size = int(len(x_train) * dataset_size)

            subset_x_train = x_train[:size]
            subset_y_train = y_train[:size]
            subset_x_test = x_test[:size]
            subset_y_test = y_test[:size]
            
            df = fit_and_validate(all_models[model], model, epochs, subset_x_train, subset_y_train, subset_x_test, subset_y_test, df)

Epoch 1/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.5592 - loss: 1.4216 - val_accuracy: 0.9107 - val_loss: 0.3130
Epoch 2/2
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9218 - loss: 0.2738 - val_accuracy: 0.9400 - val_loss: 0.1787
Test loss: 0.14548340439796448
Test accuracy: 0.9577999711036682
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
Epoch 1/2
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9442 - loss: 0.1836 - val_accuracy: 0.9656 - val_loss: 0.1133
Epoch 2/2
127/127 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9575 - loss: 0.1438 - val_accuracy: 0.9711 - val_loss: 0.0918
Test loss: 0.0832858681678772
Test accuracy: 0.975600004196167
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Epoch 1/2
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9615 - loss: 0.1242 - val_accuracy: 0.9676 - val_loss: 0.1014
Epoch 2/2
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9690 - loss: 0.0974 - val_accuracy: 0.9757 - val_loss: 0.0830
Test loss: 0.070794

In [ ]:
df

In [ ]:
plt.scatter(df.time_to_train, df.dataset_size])

In [ ]:
plt.scatter(df.time_to_train, df.accuracy)